# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, r2_score

In [4]:
from explainerdashboard.explainers import *
from explainerdashboard.dashboards import *
from explainerdashboard.datasets import *

# RandomForestRegressor

In [7]:
X_train, y_train, X_test, y_test = titanic_fare()
train_names, test_names = titanic_names()

model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestRegressionBunch(model, X_test, y_test, r2_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names)

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False,
                        shadow_trees=True)
db.run(8051)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating shap values...
Calculating importances...
Generating shadow trees...
Running Model Explainer on http://localhost:8051
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2019 14:29:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "GET /assets/bootstrap.css?m=1573382556.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "GET /assets/stylesheet.css?m=1573382556.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "GET /assets/dash_bootstrap_components.min.js?m=1573382556.0 HTTP/1.1" 304 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "GET /_favicon.ico?v=1.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2019 14:29:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - -

# RandomForestClassifier:

In [8]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)


explainer = RandomForestClassifierBunch(model, X_test, y_test, roc_auc_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        shadow_trees=True)
db.run(8052)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating prediction probabilities...
Calculating importances...
Calculating shap interaction values...
Generating shadow trees...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Dec/2019 11:56:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 11:56:31] "POST /_dash-update-component

In [10]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        shadow_trees=True)
db.run(8052)

Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Dec/2019 12:06:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2019 12:06:53] "POST /_dash-update-component

# XGBClassifier:

In [5]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

explainer = TreeModelClassifierBunch(xgb_model, X_test, y_test, roc_auc_score, 
                                   cats=['Sex', 'Cabin', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False)
db.run(8052)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Calculating shap values...
Generating shap TreeExplainer...
Calculating predictions...
Calculating prediction probabilities...
Calculating importances...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2019 11:52:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:46] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566138672 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1566138674 HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2019 11:52:47] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566138668 HTTP/1.1